In [1]:
import requests
import time
import tqdm
import pandas as pd
import os
import json
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import re
import time
from pathlib import Path

fp_data = Path.cwd() / "data"  
# Use the Path object to actually create the subfolder
Path.mkdir(fp_data, exist_ok=True) 

In [2]:
# Scrabing data about books on the 5 social sciences from Goodreads.com by looping over the URLs 

html=[]

subjects = ['economics', 'sociology', 'anthropology', 'psychology', 'political+science']

for page in tqdm.tqdm(range(1,101)):
    for subject in subjects:
        url = f'https://www.goodreads.com/search?page={page}&utf8=%E2%9C%93&query={subject}'
        response = requests.get(url, headers={'name':'Anna Nielsen', 'Universitet': 'UNI of Copenhagen','Message': 'Data collecting for exam project in Social Data science', 'email':'djb713@alumni.ku.dk'})
        soup = BeautifulSoup(response.content, 'lxml')
        a_links = soup.find_all('a', class_='bookTitle')
        html.append(a_links)
    
        time.sleep(5)
    
html2 = [item for sublist in html for item in sublist]

liste2=[]
for i in html2:
    a_links2=i.get('href')
    liste2.append('https://www.goodreads.com'+a_links2)
    
len(liste2)

100%|██████████| 100/100 [57:33<00:00, 34.54s/it]


9979

In [5]:
# Dropping dublicates from the list of URLs above 

list_nodup = []

for item in liste2:
    if item not in list_nodup:
        list_nodup.append(item)

len(list_nodup)      

9979

In [ ]:
# KUN ØKONOMI 

html=[]


for page in tqdm.tqdm(range(1,101)):
    url = f'https://www.goodreads.com/search?page={page}&utf8=%E2%9C%93&query=economics'
    response = requests.get(url, headers={'name':'Anna Nielsen', 'Universitet': 'UNI of Copenhagen','Message': 'Data collecting for exam project in Social Data science', 'email':'djb713@alumni.ku.dk'})
    soup = BeautifulSoup(response.content, 'lxml')
    a_links = soup.find_all('a', class_='bookTitle')
    html.append(a_links)
    
    time.sleep(5)

listlist=[]
for i in html:
    a_links2=i.get('href')
    listlist.append('https://www.goodreads.com'+a_links2)
    
len(listlist)

 68%|██████▊   | 68/100 [08:27<03:39,  6.85s/it]

In [6]:
def log(response, logfile, output_path=fp_data):
    # Create the local folder if it doesn't exist
    if not os.path.exists(fp_data):
        os.makedirs(fp_data)
    
    # Construct the full path for the log file within the fp_data folder
    log_filepath = os.path.join(fp_data, logfile)
    
    # Open or create the csv file
    if os.path.isfile(log_filepath):
        log_mode = 'a'
    else:
        log_mode = 'w'
        header = ['timestamp', 'status_code', 'length', 'output_file']
        with open(log_filepath, 'w') as log:
            log.write(';'.join(header) + '\n')
        
    # Gather log information
    status_code = response2.status_code
    timestamp = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))
    length = len(response2.text)
    
    # Append the gathered log information
    with open(log_filepath, 'a') as log:
        log.write(f'{timestamp};{status_code};{length};{log_filepath}\n')

In [7]:
#Storing data on each book in a dataframe 'books_df'. 
#We loop over 5 attemps to try getting the information, else we drop the book 
#With try and except we drop books with errors in loading information (often because the book don't have the variable we are interested in)

books_df = pd.DataFrame()

loops = 0

logfile = 'log.csv'

for url in tqdm.tqdm(listlist):
    
    loops += 1
    #print(loops)
        
    try:
        attempts = 5
        for _ in range(attempts):
            
            response2 = requests.get(url, headers={'name':'Barbara Bille Tagmose', 'Universitet': 'UNI of Copenhagen','Besked': 'Data collecting for exam project in Social Data science', 'email':'wjl145@alumni.ku.dk'})
            time.sleep(5)
            suppe = BeautifulSoup(response2.content, 'lxml') 

            #Finder elementer i supper 
            bogtitel_ = suppe.find_all('h1', class_='Text Text__title1')
            stjerner_ = suppe.find_all('div', class_='RatingStatistics__rating')
            metastatistik_ = suppe.find_all('div', class_='RatingStatistics__meta')
            forfattere_ = suppe.find_all('div', class_='ContributorLinksList')
            beskrivelse_ = suppe.find_all('div', class_='BookPageMetadataSection__description')
            plublishdetails_ = suppe.find_all('div', class_='FeaturedDetails')

            #Laver tomme lister 
            bogtitel = []
            Stjerner = []
            metastatistik = []
            nratings = []
            nreviews =[]
            forfatter = []
            beskrivelse = []
            plublishdetails = []
            pages = []
            published = []

            #Laver variable i loops 
            for i in bogtitel_:
                bogtitel = str(i.text)

            for i in stjerner_:
                Stjerner = float(i.text)

            for i in metastatistik_:
                metastatistik = i.find_all('span')

            for idx, r in enumerate(metastatistik):
                if idx == 0:
                    ratingsstr = str(r)
                    nratings = int(re.sub(r'\D+','',ratingsstr))
                else:
                    reviewsstr = str(r)
                    nreviews = int(re.sub(r'\D+','',reviewsstr))

            for i in forfattere_:
                forfatter.append(i.text)

            for i in beskrivelse_:
                beskrivelse = str(i.text)

            for i in plublishdetails_:
                plublishdetails = i.find_all('p')

            for idx, i in enumerate(plublishdetails):
                if idx < len(plublishdetails)-1:
                    pagesstr = str(i.text)
                    pages = int(re.sub(r'\D+','',pagesstr))
                else:
                    publishedstr = str(i.text)
                    publisheddig = str(re.sub(r'\D+','',publishedstr))
                    match = re.search(r'.{4}$', publisheddig)
                    published = int(match.group())

            #Laver om til et pandas dataframe 

            data = {
            'Title': bogtitel,
            'Author': forfatter,
            'Avg_ratings': Stjerner,
            'nratings': nratings,
            'nreviews': nreviews,
            'Description': beskrivelse,
            'Pages': pages,
            'Published_year': published

            }
            #print(data)

            result_df = pd.DataFrame(data)
            
            log(response2,logfile)


            if not result_df.empty:
                books_df = pd.concat([books_df,result_df], axis=0, ignore_index=True) #Append to the rest of the data
                
                if loops %100 == 0:
                    books_df.to_csv(fp_data/'bookssave100.csv')
                
                break    
            #else:
                #print('fejl')
        #else:
            #print('overskredet')
    except Exception as e:
        print(url)
        print(e)
        continue
        


  0%|          | 45/9979 [11:41<43:02:42, 15.60s/it]


KeyboardInterrupt: 

In [25]:
books_df


,Title,Author,Avg_ratings,nratings,nreviews,Description,Pages,Published_year
0,Poor Economics: A Radical Rethinking of the Wa...,"Abhijit V. Banerjee, Esther Duflo",4.28,20752,1888,Why do the poor borrow to save? Why do they mi...,320,2011
1,Misbehaving: The Making of Behavioral Economics,Richard H. Thaler,4.16,20440,1523,Nobel laureate Richard H. Thaler has spent his...,358,2016
2,Good Economics for Hard Times: Better Answers ...,"Abhijit V. Banerjee, Esther Duflo",4.24,12174,1262,Figuring out how to deal with today's critical...,417,2019
